In [1]:
import pandas as pd
import pickle
from pymongo import MongoClient
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np

In [2]:
collection = 'covid_county_formatted'

db = MongoClient("lattice-100", 27018)
no_of_records = 1064950

pipeline = [
    {
        "$sample": {"size": no_of_records // 5}
    }
]

cursor = db.sustaindb[collection].aggregate(pipeline)
df_original = pd.DataFrame(list(cursor))

In [3]:
df = df_original[["GISJOIN", "cases"]]
df

,GISJOIN,cases
0,G4600910,1
1,G5500750,1
2,G1301090,1
3,G1800870,10
4,G4804850,45
...,...,...
212985,G3000790,0
212986,G2801450,2
212987,G4000810,4
212988,G4803310,7


## Aggregate on 'cases' for each GISJoin

In [4]:
df1 = pd.DataFrame(df.groupby("GISJOIN")["cases"].sum())
dfX = df1['cases']
df1

,cases
GISJOIN,
G0100010,1443
G0100030,4564
G0100050,372
G0100070,350
G0100090,960
...,...
G5600370,698
G5600390,676
G5600410,373


In [5]:
dfX = np.array(dfX).reshape(-1, 1)

In [6]:
kmeans = KMeans(n_clusters=56, random_state=0).fit(dfX)
df_predict = kmeans.fit_predict(dfX)
centroids = kmeans.cluster_centers_
df1['cluster_id'] = df_predict
df1

,cases,cluster_id
GISJOIN,,
G0100010,1443,28
G0100030,4564,8
G0100050,372,51
G0100070,350,51
G0100090,960,16
...,...,...
G5600370,698,55
G5600390,676,55
G5600410,373,51


In [7]:
len(centroids)

56

In [8]:
cluster_ids_list = df1['cluster_id']
centroids_column = []

for id in cluster_ids_list:
    centroids_column.append(centroids[id][0])
    
print(len(centroids_column))

3115


In [9]:
df1['centroid'] = centroids_column

In [10]:
df1['distance'] = abs(df1['cases'] - df1['centroid'])
df1

,cases,cluster_id,centroid,distance
GISJOIN,,,,
G0100010,1443,28,1445.045455,2.045455
G0100030,4564,8,4516.535714,47.464286
G0100050,372,51,366.832618,5.167382
G0100070,350,51,366.832618,16.832618
G0100090,960,16,1057.492857,97.492857
...,...,...,...,...
G5600370,698,55,685.379310,12.620690
G5600390,676,55,685.379310,9.379310
G5600410,373,51,366.832618,6.167382


In [11]:
df2 = df1[["cluster_id", "distance"]].reset_index()
df2

,GISJOIN,cluster_id,distance
0,G0100010,28,2.045455
1,G0100030,8,47.464286
2,G0100050,51,5.167382
3,G0100070,51,16.832618
4,G0100090,16,97.492857
...,...,...,...
3110,G5600370,55,12.620690
3111,G5600390,55,9.379310
3112,G5600410,51,6.167382
3113,G5600430,48,18.229064


### Tag Parent GISJoins

In [30]:
df2_gr = df2.groupby('cluster_id')['distance'].min().reset_index()
parents = []
for i, row in df2_gr.iterrows():
    cluster_id = row['cluster_id']
    distance = row['distance']
    
    cluster_id_match = df2[df2['cluster_id'] == cluster_id]
    distance_match = cluster_id_match[cluster_id_match['distance'] == distance].reset_index()['GISJOIN']
    parent = list(distance_match)[0] # select only one GISJOIN
    parents.append(parent)
    
print(len(set(parents)))

56


In [13]:
df_clusters = df2_gr.rename(columns={'distance': 'min_distance'})
df_clusters['max_distance'] = df2.groupby('cluster_id')['distance'].max().reset_index()['distance']
print(df_clusters.shape)
df_clusters.head()

(56, 3)


,cluster_id,min_distance,max_distance
0,0,2.057971,128.942029
1,1,557.500000,557.500000
2,2,8.500000,475.500000
3,3,0.000000,0.000000
4,4,39.666667,246.333333


In [14]:
for parent in parents:
    df2.loc[df2.GISJOIN == parent, "is_parent"] = 1

df2 = df2.fillna(0)
df2.head()

,GISJOIN,cluster_id,distance,is_parent
0,G0100010,28,2.045455,0.0
1,G0100030,8,47.464286,0.0
2,G0100050,51,5.167382,0.0
3,G0100070,51,16.832618,0.0
4,G0100090,16,97.492857,0.0


In [15]:
df3 = df2

for i, row in df2.iterrows():
    i_distance = row['distance']
    cluster_id = row['cluster_id']
    gis_join = row['GISJOIN']
    max_distance = df_clusters[df_clusters['cluster_id'] == cluster_id]['max_distance'].item()
    min_distance = df_clusters[df_clusters['cluster_id'] == cluster_id]['min_distance'].item()
    if max_distance == min_distance:
        frac = 0
    else:
        frac = (i_distance - min_distance)/(max_distance - min_distance)
    df3.loc[df3.GISJOIN == gis_join, "frac_distance"] = frac

In [16]:
sample_min = 0.05
sample_max = 0.25

df3['sample_percent'] = sample_min + (sample_max - sample_min) * df3['frac_distance']
df3

,GISJOIN,cluster_id,distance,is_parent,frac_distance,sample_percent
0,G0100010,28,2.045455,0.0,0.011364,0.052273
1,G0100030,8,47.464286,0.0,0.158088,0.081618
2,G0100050,51,5.167382,0.0,0.104167,0.070833
3,G0100070,51,16.832618,0.0,0.347192,0.119438
4,G0100090,16,97.492857,0.0,0.905660,0.231132
...,...,...,...,...,...,...
3110,G5600370,55,12.620690,0.0,0.162695,0.082539
3111,G5600390,55,9.379310,0.0,0.119615,0.073923
3112,G5600410,51,6.167382,0.0,0.125000,0.075000
3113,G5600430,48,18.229064,0.0,0.336185,0.117237


In [17]:
unique_parents1 = list(df3[df3['is_parent'] == 1]['GISJOIN'].unique())
print(len(unique_parents1))

56


In [18]:
df_cluster_parents = df3[df3['is_parent'] == 1][['cluster_id', 'GISJOIN']].sort_values('cluster_id').reset_index()
del df_cluster_parents['index']
cluster_parent_map = {}

for i, row in df_cluster_parents.iterrows():
    cluster_parent_map[row['cluster_id']] = row['GISJOIN']

In [19]:
cluster_parent_map.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55])

In [31]:
cluster_parent_map.values()

dict_values(['G2700270', 'G0400130', 'G4900490', 'G0600370', 'G4800290', 'G1600750', 'G3400070', 'G0600650', 'G4800270', 'G2101110', 'G4802010', 'G0600590', 'G0800410', 'G4201010', 'G1800950', 'G4600910', 'G1700270', 'G2100670', 'G0600730', 'G0600010', 'G1200860', 'G3900350', 'G5300050', 'G4801570', 'G1901670', 'G4900350', 'G0800690', 'G1301170', 'G0500330', 'G1600810', 'G3600290', 'G3900490', 'G4500790', 'G3000310', 'G5500090', 'G1300210', 'G3601190', 'G4001090', 'G1200570', 'G1200710', 'G1200990', 'G0100790', 'G3600590', 'G4804390', 'G4200710', 'G3901510', 'G5300350', 'G3701190', 'G0500890', 'G4803750', 'G3400290', 'G4701710', 'G4200910', 'G4500450', 'G4500310', 'G1801030'])

In [38]:
child_df = df3[df3['is_parent'] == 0]

## Find Children for each Parent

In [41]:
parent_child_map = {}
i = 1
for cluster_id, parent in cluster_parent_map.items():
    children = list(child_df[child_df['cluster_id'] == cluster_id]['GISJOIN'])
    parent_child_map[parent] = children
    print(f'{i}. {parent}: {len(children)}')
    i += 1

1. G2700270: 67
2. G0400130: 1
3. G4900490: 5
4. G0600370: 0
5. G4800290: 2
6. G1600750: 193
7. G3400070: 13
8. G0600650: 1
9. G4800270: 26
10. G2101110: 8
11. G4802010: 0
12. G0600590: 0
13. G0800410: 7
14. G4201010: 2
15. G1800950: 35
16. G4600910: 509
17. G1700270: 129
18. G2100670: 19
19. G0600730: 1
20. G0600010: 5
21. G1200860: 0
22. G3900350: 2
23. G5300050: 44
24. G4801570: 14
25. G1901670: 183
26. G4900350: 0
27. G0800690: 32
28. G1301170: 25
29. G0500330: 69
30. G1600810: 324
31. G3600290: 7
32. G3900490: 7
33. G4500790: 19
34. G3000310: 53
35. G5500090: 13
36. G1300210: 37
37. G3601190: 0
38. G4001090: 3
39. G1200570: 2
40. G1200710: 5
41. G1200990: 0
42. G0100790: 161
43. G3600590: 0
44. G4804390: 0
45. G4200710: 10
46. G3901510: 14
47. G5300350: 82
48. G3701190: 1
49. G0500890: 406
50. G4803750: 47
51. G3400290: 4
52. G4701710: 230
53. G4200910: 3
54. G4500450: 5
55. G4500310: 53
56. G1801030: 181


In [42]:
pickle.dump(parent_child_map, open('pickles/parent_child_map.pkl', 'wb'))

## Find Parent for each Child

In [20]:
children = list(df3[df3['is_parent'] == 0]['GISJOIN'])

child_parent_map = {}
for child in children:
    cluster_id = df3[df3['GISJOIN'] == child]['cluster_id'].item()
    parent = cluster_parent_map[cluster_id]
    child_parent_map[child] = parent
    
# child_parent_map.values().unique()

In [21]:
len(child_parent_map.keys())

3059

In [22]:
child_parent_map['G5600150']

'G0500890'

In [33]:
values = child_parent_map.values()
len(set(values))

47

In [24]:
# serialize child_parent_map
pickle.dump(child_parent_map, open('pickles/covid_child_parent_map.pkl', 'wb'))

In [25]:
# Write child parent map to csv
children = []
parents = []
for child, parent in child_parent_map.items():
    children.append(child)
    parents.append(parent)
    
df_child_parent = pd.DataFrame(zip(children, parents), columns=['child', 'parent'])
# df_child_parent.to_csv('ucc-21/covid_child_parent_map.csv', index=False)

In [26]:
# Child Parent Map Sanity Check
unique_children = list(set(children))
unique_parents2 = list(set(parents))

print(len(unique_children))
print(len(unique_parents2))

3059
47


In [28]:
len(set(unique_parents1) - set(unique_parents2))

9

In [29]:
df_final = df3
df_final.to_csv('./clusters-covid.csv')